## Thomson Numerical Optimization

In [ ]:
import torch
import matplotlib.pyplot as plt
from IPython.display import clear_output
from smale7.utils.visualization import plot_thomson
from matplotlib.animation import FuncAnimation, PillowWriter


# Parameters
N = 3  # Number of points
lr = 0.01  # Learning rate
num_steps = 1_000  # Number of optimization steps

# Initialize points randomly on the sphere
points = torch.randn(N, 3)  # Create without gradients
points /= points.norm(dim=1, keepdim=True)  # Normalize to lie on the sphere
points.requires_grad_()  # Enable gradients

# Optimizer
optimizer = torch.optim.Adam([points], lr=lr)

# Frames
frames = []

# Optimization loop
for step in range(num_steps):
    # Zero the gradients
    optimizer.zero_grad()

    # Compute the energy
    energy = 0
    for i in range(N):
        for j in range(i + 1, N):
            distance = torch.norm(points[i] - points[j])
            energy += 1 / distance  # Coulomb potential

    # Backpropagate to compute gradients
    energy.backward()

    # Project gradients onto the tangent space of the sphere
    with torch.no_grad():
        grad = points.grad
        tangent_grad = grad - (grad * points).sum(dim=1, keepdim=True) * points
        points.grad.copy_(tangent_grad)  # Replace the gradient with the tangent component

    # Take an optimizer step
    optimizer.step()

    # Reproject points back onto the sphere
    with torch.no_grad():
        points.data /= points.data.norm(dim=1, keepdim=True)

    # Visualization every 10 steps
    if step % 10 == 0 or step == num_steps - 1:
        print(f"Step {step}, Energy: {energy.item()}")
        clear_output(wait=True)  # Clear previous plot
        fig = plt.figure(figsize=(6, 6))
        ax = fig.add_subplot(111, projection='3d')
        plot_thomson(points.detach(), ax, N)
        plt.title(f"N = {N}  Step = {step}  Energy = {energy.item():.4f}")
        plt.show()
        frames.append(fig)  # Save the current frame
        plt.close(fig)  # Close the figure to save memory

    # Save the frames as a GIF
    fig, ax = plt.subplots()
    gif_writer = PillowWriter(fps=10)
    ani = FuncAnimation(fig, lambda i: frames[i], frames=len(frames))
    ani.save("optimization.gif", writer=gif_writer)

    print("GIF saved as optimization.gif")

# Final result
print("Optimized Points:")
print(points)


In [ ]:
import torch
import matplotlib.pyplot as plt
from IPython.display import clear_output
from smale7.utils.visualization import plot_thomson
from matplotlib.animation import FuncAnimation, PillowWriter

%load_ext autoreload
%autoreload 2


def get_frames(N):
    # Parameters
    lr = 0.001  # Learning rate
    num_steps = 1000  # Number of optimization steps

    # Initialize points randomly on the sphere
    points = torch.randn(N, 3)  # Create without gradients
    points /= points.norm(dim=1, keepdim=True)  # Normalize to lie on the sphere
    points.requires_grad_()  # Enable gradients

    # Optimizer
    optimizer = torch.optim.Adam([points], lr=lr)

    # Initialize list for frames
    frames = []

    # Optimization loop
    for step in range(num_steps):
        # Zero the gradients
        optimizer.zero_grad()

        # Compute the energy
        energy = 0
        for i in range(N):
            for j in range(i + 1, N):
                distance = torch.norm(points[i] - points[j])
                energy += 1 / distance  # Coulomb potential

        # Backpropagate to compute gradients
        energy.backward()

        # Project gradients onto the tangent space of the sphere
        with torch.no_grad():
            grad = points.grad
            tangent_grad = grad - (grad * points).sum(dim=1, keepdim=True) * points
            points.grad.copy_(tangent_grad)  # Replace the gradient with the tangent component

        # Take an optimizer step
        optimizer.step()

        # Reproject points back onto the sphere
        with torch.no_grad():
            points.data /= points.data.norm(dim=1, keepdim=True)

        # Visualization every 10 steps
        if step % 10 == 0 or step == num_steps - 1:
            print(f"Step = {step}, Energy = {energy.item()}")
            # clear_output(wait=True)  # Clear previous plot
            frames.append((step, points.detach().clone(), energy.detach().clone()))

        if step % 5000 == 0 or step == num_steps - 1:
            clear_output(wait=True)  # Clear previous plot
            fig = plt.figure(figsize=(6, 6))
            ax = fig.add_subplot(111, projection='3d')
            plot_thomson(ax, points.detach(), N)
            plt.title(f"N = {N}, Step = {step}, Energy = {energy.item():.4f}")
            plt.tight_layout()
            plt.show()
            plt.close(fig)  # Close the figure to save memory

    print(f"Step = {step}, Energy = {energy.item()}")
    return frames


frames_dict = {}
for N in [100]:
    frames_dict[N] = get_frames(N)
    break

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# Save the frames as a GIF after the loop
fig = plt.figure(figsize=(18, 6))  # Horizontal layout for side-by-side plots
ax1 = fig.add_subplot(131, projection='3d')  # Left subplot
ax2 = fig.add_subplot(132, projection='3d')  # Right subplot
ax3 = fig.add_subplot(133, projection='3d')  # Right subplot

# Manually set positions for the subplots
ax1.set_position([0.02, 0.1, 0.3, 0.8])  # Left subplot
ax2.set_position([0.36, 0.1, 0.3, 0.8])  # Middle subplot
ax3.set_position([0.70, 0.1, 0.3, 0.8])  # Right subplot

def update_frame(i):

    # Update left plot
    step, points, energy = frames4[i]
    ax1.clear()
    plot_thomson(ax1, points, N=4)
    ax1.set_title(f"N = {4}, Step = {step}, Energy = {energy.item():.4f}", y=1.1)


    # Update right plot (identical copy)
    step, points, energy = frames5[i]
    ax2.clear()
    plot_thomson(ax2, points, N=5)
    ax2.set_title(f"N = {5}, Step = {step}, Energy = {energy.item():.4f}", y=1.1)

    # Update right plot (identical copy)
    step, points, energy = frames6[i]
    ax3.clear()
    plot_thomson(ax3, points, N=6)
    ax3.set_title(f"N = {6}, Step = {step}, Energy = {energy.item():.4f}", y=1.1)


# Create the animation
ani = FuncAnimation(fig, update_frame, frames=len(frames4))
ani.save("optimization_side_by_side.gif", writer=PillowWriter(fps=10), dpi=150)
print("GIF saved as optimization_side_by_side.gif")

# Final result
print("Optimized Points:")
print(points)


In [ ]:
# # Save the frames as a GIF after the loop
# fig = plt.figure(figsize=(6, 6))
# ax = fig.add_subplot(111, projection='3d')

# def update_frame(i):
#     step, points, energy = frames5[i]
#     ax.clear()
#     plot_thomson(ax, points, N)
#     ax.set_title(f"N = {N}, Step = {step}, Energy = {energy.item():.4f}", y=1.02)
    
# ani = FuncAnimation(fig, update_frame, frames=2)
# ani.save("optimization.gif", writer=PillowWriter(fps=10), dpi=150)
# print("GIF saved as optimization.gif")

# # Final result
# print("Optimized Points:")
# print(points)